In [1]:
import os
import sys

import fine as fn
import fine.IOManagement.xarrayIO as xrIO

%load_ext autoreload
%autoreload 2

# How to save an energy system model instance and set it back up? 

**Xarray and NetCDF files to the rescue!** The data contained within an Energy System Model (ESM) instance and the optimization results is vast and complex. Saving it directly is not possible. It can, however, be saved as a NetCDF file which supports complex data structures. 

#### What exactly is NetCDF? 
NetCDF (Network Common Data Format) is a set of software libraries and machine-independent data formats that support the creation, access, and sharing of array-oriented scientific data. It is also a community standard for sharing scientific data. 

#### Python modules that support working with NetCDF files:
1. netcdf4-python: Official Python interface to netCDF files
2. PyNIO: To access different file formats such as netCDF, HDF, and GRIB
3. xarray: Based on NumPy and pandas

Note: xarray module is used here. 

For our use case, the following functionalities are provided: 
* Conversion of ESM instance to xarray dataset. Additionally, possible to save this dataset as NetCDF file in a desired folder, with a desired file name. 
* Conversion of xarray dataset/saved NetCDF file back to ESM instance.

#### High-level structure of the data: 

<img src="overall_structure.png" style="width: 1000px;"/>


#### Structure of xarray dataset - For a non-transmission component: 

<img src="non_transmission.png" style="width: 1000px;"/>

#### Structure of xarray dataset - For a transmission component: 

<img src="transmission.png" style="width: 1000px;"/>


## Conversion of ESM instance to xarray dataset and saving it as a NetCDF file

#### STEP 1. Set up your  ESM instance 

In [2]:
from getModel import getModel

esM = getModel()
esM.optimize()

Using license file C:\Users\ra.maier\gurobi.lic
Academic license - for non-commercial use only
Read LP format model from file C:\Users\RAEF3F~1.MAI\AppData\Local\Temp\tmpwxqr6him.pyomo.lp
Reading time = 0.03 seconds
x115: 99 rows, 75 columns, 243 nonzeros
Changed value of parameter QCPDual to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter Threads to 3
   Prev: 0  Min: 0  Max: 1024  Default: 0
Parameter logfile unchanged
   Value:   Default: 
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 99 rows, 75 columns and 243 nonzeros
Model fingerprint: 0x7500b870
Coefficient statistics:
  Matrix range     [3e-01, 2e+03]
  Objective range  [7e-03, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+09]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 49 rows and 41 columns
Presolve time: 0.00s
Presolved: 50 rows, 34 columns, 168 nonzeros

Iteration    O

#### STEP 2. Conversion to xarray datasets and saving as NetCDF file
You can convert the esM to xarray datasets with `esm_to_datasets` and access Input, Parameters or Result.


In [3]:
esm_datasets = xrIO.writeEnergySystemModelToDatasets(esM)

In [4]:
esm_datasets["Input"]["Sink"]["Industry site"][
    "ts_operationRateFix"
].to_dataframe().unstack()

ts_operationRateFix                 
space ElectrolyzerLocation IndustryLocation
time                                       
0                      0.0       13140000.0
1                      0.0       13140000.0
2                      0.0       13140000.0
3                      0.0       13140000.0

In [7]:
esm_datasets["Results"][0]["SourceSinkModel"]["Electricity market"]

<xarray.Dataset>
Dimensions:                    (space: 2, time: 4)
Coordinates:
  * space                      (space) <U20 'ElectrolyzerLocation' 'IndustryL...
  * time                       (time) int64 0 1 2 3
Data variables: (12/16)
    NPVcontribution            (space) float64 1.52e+06 0.0
    TAC                        (space) float64 1.52e+06 0.0
    capacity                   (space) float64 nan nan
    capexCap                   (space) float64 nan nan
    capexIfBuilt               (space) float64 nan nan
    commissioning              (space) float64 nan nan
    ...                         ...
    isBuilt                    (space) float64 nan nan
    operation                  (space) float64 7.509e+07 nan
    opexCap                    (space) float64 nan nan
    opexIfBuilt                (space) float64 nan nan
    opexOp                     (space) float64 0.0 nan
    operationVariablesOptimum  (time, space) float64 1.877e+07 nan ... nan
Attributes: (12/15)
    NPVcontribution:  [1 Euro]
    TAC:              [1 Euro/a]
    capacity:         [kW$_{el}$]
    capexCap:         [1 Euro/a]
    capexIfBuilt:     [1 Euro/a]
    commissioning:    [kW$_{el}$]
    ...               ...
    invest:           [1 Euro]
    isBuilt:          [-]
    operation:        [kW$_{el}$*h]
    opexCap:          [1 Euro/a]
    opexIfBuilt:      [1 Euro/a]
    opexOp:           [1 Euro/a]

In [8]:
esm_datasets["Parameters"]

<xarray.Dataset>
Dimensions:  ()
Data variables:
    *empty*
Attributes: (12/14)
    locations:                  {'ElectrolyzerLocation', 'IndustryLocation'}
    commodities:                {'hydrogen', 'electricity'}
    commodityUnitsDict:         {'electricity': 'kW$_{el}$', 'hydrogen': 'kW$...
    numberOfTimeSteps:          4
    hoursPerTimeStep:           2190
    startYear:                  0
    ...                         ...
    stochasticModel:            False
    costUnit:                   1 Euro
    lengthUnit:                 km
    verboseLogLevel:            1
    balanceLimit:               None
    lowerBound:                 False

Or save it directly to NetCDF with `esm_to_netcdf`:

In [9]:
_ = xrIO.writeEnergySystemModelToNetCDF(
    esM, outputFilePath="my_esm.nc", overwriteExisting=True
)

#### STEP 3. Load esM from NetCDF file or xarray datasets

You can load an esM from file with `netcdf_to_esm`.

In [10]:
esm_from_netcdf = xrIO.readNetCDFtoEnergySystemModel("my_esm.nc")

In [11]:
esm_from_netcdf.getComponentAttribute("Industry site", "operationRateFix")

space,ElectrolyzerLocation,IndustryLocation
time,,
0,0.0,13140000.0
1,0.0,13140000.0
2,0.0,13140000.0
3,0.0,13140000.0


Or from datasets with `datasets_to_esm`.

In [12]:
esm_from_datasets = xrIO.convertDatasetsToEnergySystemModel(esm_datasets)

In [13]:
esm_from_datasets.getComponentAttribute("Industry site", "operationRateFix")

space,ElectrolyzerLocation,IndustryLocation
time,,
0,0.0,13140000.0
1,0.0,13140000.0
2,0.0,13140000.0
3,0.0,13140000.0


In [15]:
esm_datasets["Results"][0]["SourceSinkModel"]["Electricity market"][
    "operationVariablesOptimum"
]

<xarray.DataArray 'operationVariablesOptimum' (time: 4, space: 2)>
array([[18771428.57142857,               nan],
       [37542857.14285714,               nan],
       [       0.        ,               nan],
       [18771428.57142857,               nan]])
Coordinates:
  * space    (space) <U20 'ElectrolyzerLocation' 'IndustryLocation'
  * time     (time) int64 0 1 2 3